In [ ]:
import numpy as np

from tifffile import imread, imwrite
from matplotlib import pyplot as plt
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.util import random_noise
from nanopyx.core.transform._le_nlm_denoising import NLMDenoising
from nanopyx.core.transform._le_fast_nlm_denoising import NLMDenoising as FastNLMDenoising

from nanopyx.core.utils.timeit import timeit2

from nanopyx.data.download import ExampleDataManager

downloader = ExampleDataManager()
image = downloader.get_ZipTiffIterator(
    "SMLMS2013_HDTubulinAlexa647", as_ndarray=True).astype(np.float32)

#%matplotlib widget

In [ ]:
# img = imread("../../SMLM.tif")
# img = random_noise(np.random.random((1, 100, 100)), mode="gaussian", mean=2, var=2, seed=3)
# patch_size = 5
# patch_distance = 11
# h = 0.1
# sigma = 2

# denoiser = NLMDenoising(testing=True, clear_benchmarks=True)
# for i in range(3):
#     denoiser.benchmark(img, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma)

In [ ]:
denoiser = NLMDenoising(testing=True)
fdenoiser = FastNLMDenoising(testing=True)
patch_size = 3
patch_distance = 10

img = (image[0] - image[0].min())/image[0].max()
sigma = 3
h = 0.8 * sigma

unt = denoiser.run(img, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, run_type="Unthreaded")
thr = denoiser.run(img, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, run_type="Threaded")
pyt = denoiser.run(img, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, run_type="Python")
cl = img

unt_fast = fdenoiser.run(img, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, run_type="Unthreaded")
thr_fast = fdenoiser.run(img, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, run_type="Threaded")
pyt_fast = fdenoiser.run(img, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, run_type="Python")
cl_fast = fdenoiser.run(img, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, run_type="OpenCL_Apple M1")

fig,axs = plt.subplots(1,9,figsize=(15,15))
axs[0].imshow(img)

# non fast
axs[1].imshow(unt)
axs[2].imshow(thr)
axs[3].imshow(pyt)
axs[4].imshow(cl)

# fast
axs[5].imshow(unt_fast)
axs[6].imshow(thr_fast)
axs[7].imshow(pyt_fast)
axs[8].imshow(cl_fast)
plt.show()


In [ ]:
# denoiser = NLMDenoising(testing=True)
# fdenoiser = FastNLMDenoising(testing=True)
# patch_size = 5
# patch_distance = 11
# h = 0.1
# sigma = 0.1
# img = (image[0] - image[0].min())/image[0].max()

# thr = denoiser.run(img, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma, run_type="Unthreaded")
